In [9]:
import numpy as np
import tensorflow as tf
from simulator.core.simulator import Simulator
from simulator import Config
from simulator.environment.task_environment import TaskEnvironment
from simulator.logger import LogOutputConsolePrint, Logger
from simulator.world_builds.box_world import BoxWorld
from simulator.reporters import TransitionReporter
from simulator.task_multiplexing import TaskMultiplexerSelectorDql
from simulator.task_multiplexing import TaskMultiplexerSelectorLocal
from simulator.task_multiplexing import TaskMultiplexerSelectorRandom
from simulator.task_multiplexing import TaskMultiplexerSelectorRemote
from simulator.task_multiplexing import TaskMultiplexerSelectorGreedy

## Set simulator behaviour

In [10]:
Logger.registerLogOutput(LogOutputConsolePrint())
np.random.seed(Config.get('random_seed'))
tf.random.set_seed(Config.get('random_seed'))

## Creating an instace of Box World

In [11]:

world = BoxWorld()
edgeNodes, mobileNodes = world.build()
edgeReward, mobileReward = world.defaultRewards()


## Creating the reporters

In [12]:
mobileReporter = TransitionReporter() 
edgeReporter = TransitionReporter() 

## Task multiplexing

In [13]:
dqlSelectorGenerator = lambda state: TaskMultiplexerSelectorDql(state, Config.get("dql_training_buffer_size"))
localSelectorGenerator = lambda state: TaskMultiplexerSelectorLocal()
remoteSelectorGenerator = lambda state: TaskMultiplexerSelectorRemote()
randomSelectorGenerator = lambda state: TaskMultiplexerSelectorRandom()
greedySelectorGenerator = lambda state: TaskMultiplexerSelectorGreedy(state == (9, ))

## Prepare and Run the Simulation

### Both dql

In [14]:
taskEnvironment = TaskEnvironment(edgeNodes, mobileNodes, 
                                  edgeSelectorGenerator= greedySelectorGenerator, 
                                  mobileSelectorGenerator= greedySelectorGenerator,
                                  edgeRewardFunction= edgeReward,
                                  mobileRewardFunction=mobileReward)
simulator = Simulator()
taskEnvironment.initialize(simulator, [mobileReporter.addTransition], [edgeReporter.addTransition])
simulator.run()

Time: 0, edge load: [6247, 0, 0]
Time: 0, edge load: [6247, 0, 0]
Time: 0, edge load: [6248, 0, 0]
Time: 0, edge load: [6248, 0, 0]
Time: 0, edge load: [6249, 0, 0]
Time: 0, edge load: [6249, 0, 0]
Time: 0, edge load: [6250, 0, 0]
Time: 0, edge load: [6250, 0, 0]
Time: 0.6031813352662093, Run Complete: Task//id:7509 size:1000000 workload:596595 nodeId:6267 arrivalTime:0.006586335266209328 hopLimit:2 powerConsumed:0.2982975 log:['local run(0.006586335266209328)']
Time: 0.6031813352662093, Run Complete: Task//id:7509 size:1000000 workload:596595 nodeId:6267 arrivalTime:0.006586335266209328 hopLimit:2 powerConsumed:0.2982975 log:['local run(0.006586335266209328)']
Time: 0.9353961601588627, Run Complete: Task//id:7513 size:1382110 workload:623207 nodeId:6285 arrivalTime:0.0740555333888627 hopLimit:2 powerConsumed:0.430670313385 log:['local run(0.0740555333888627)']
Time: 0.9353961601588627, Run Complete: Task//id:7513 size:1382110 workload:623207 nodeId:6285 arrivalTime:0.0740555333888627 

## Report delay

In [15]:
print("mobile delay: " + str(mobileReporter.averageDelay()))
print("mobile power consumed: " + str(mobileReporter.averagePowerConsumed()))

mobile delay: 3.024390262330601
mobile power consumed: 0.6531383489400012
